# Sparsity in activations and gradients for VGG16

In [ ]:
% matplotlib inline

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

log_path = '/home/rbodo/.snntoolbox/data/imagenet/vgg16_trained'
data = tf.train.summary_iterator(log_path)

In [ ]:
for v in data.summary.value:
    if 'out' in v.tag:
        activations = v.simple_value
    if 'grad' in v.tag:
        gradients = v.simple_value

In [ ]:
def get_sparsity(data):
    return 1 - np.count_nonzero(data) / np.size(data)  # Do this per epoch and layer!

In [ ]:
activation_sparsity = [get_sparsity(a) for a in activations]
gradient_sparsity = [get_sparsity(g) for g in gradients]

In [ ]:
for a in activation_sparsity:
    plt.plot(a)
plt.show()